---------------------------------------------------------------------------------------------
#### INITIALIZATION 1: REGULAR K MEANS
    1. Randomly picking initial centers


#### INITIALIZATION 2: K++ MEANS
    1. k lists for k centroids 
    2. Basically, find the largest distnace from the initial point, pick that as your centroid
    3. Continue to populate 

#### IMPORTANT POINTS
    1. Clusters need to work from 2-10
    2. Plot Objective Function value vs. Number Of Clusters, k
    3. Plot Objective Function TWICE under EACH STRATEGY (each start under different 
    initialization,  to smooth out any discrepancies from the first run)

In [ ]:
import scipy.io
import numpy as np
import random # for alternative approach for #1
import matplotlib.pyplot as plt, mpld3
from matplotlib import style

style.use('ggplot')

In [ ]:
Numpyfile = scipy.io.loadmat('AllSamples.mat')

# Return as numpy array
coordinatesArr = Numpyfile['AllSamples']
#coordinatesArr

### INITIALIZATION 1:  K MEANS

In [ ]:
class K_Means1:
    # Constructor
    def __init__(self, k =3, tolerance = 0.0001, max_iterations = 500):
        self.k = k
        self.tolerance = tolerance
        self.max_iterations = max_iterations
#-----------------------------------------------------------------------------------------------------------------    
    # Distance Metric we use
    def Euclidean_distance(feat_one, feat_two):
        squared_distance = 0
        
        # Lengths of two features must be the same
        for i in range(len(feat_one)):
            squared_distance += (feat_one[i] - feat_two[i])**2
        
        ed = sqrt(squared_distance)
        return ed;
#-----------------------------------------------------------------------------------------------------------------    
    # INITIALIZATION 1: Initialize Centroids Randomly
    def centroidInit_Init1(self, coordinatesArr):
        self.centroids = {}
        self.objectivelst = []
        self.flatList = []
        
        #(!!!)
        self.objectiveDict = {}
        
        yValueObjective = 0
            
        # RANDOMNESS LIBRARY METHOD: STEP 1, Alternative Way to Randomize Initial Centroids, using Python Randomness Library
        for i in range(self.k):
            self.centroids[i] = random.choice(coordinatesArr)
            
        # STEP 2: Begin Iterations
        for i in range(self.max_iterations):
            
            # Dictionary to store points depending on {classification: coordinates}
            self.classes = {}
            
            #(!!!) localSum to sum all the values 
            localSum = 0 
            
            # Initialize Null Dictionary for classes and objective dictionary values
            for i in range(self.k):
                self.classes[i] = []
                self.objectiveDict[i] = []
            
            # STEP 3: Calculate Euclidean Distance Between Point And Cluster
            for points in coordinatesArr: 
                # 'points' have (x, y) form, calculate distance with respect to EACH centroid
                distances = [np.linalg.norm(points - self.centroids[centroid]) for centroid in self.centroids]
                
                # Take the MINIMUM of the distance of EVERY SINGLE POINT respective to that node
                classification = distances.index(min(distances))
                self.classes[classification].append(points)

            previous = dict(self.centroids)

            # STEP 4: Recalcualte Centroids For Next Run
            for classification in self.classes:
                self.centroids[classification] = np.average(self.classes[classification], axis = 0)
                
            isOptimal = True
            
            # STEP 5: Break Out Of Main Loop If Results Are Optimal
            for centroid in self.centroids:
                original_centroid = previous[centroid]
                curr = self.centroids[centroid]
                
                if np.sum((curr - original_centroid)/original_centroid * 100.00) > self.tolerance:
                    isOptimal = False
                    
            if isOptimal == True:
                break
                
        # STEP 6: Plot Objective Function For Each k
        totalSum = 0
        
        # Iterate through every class 
        for classification in self.classes:
            
            # (!!!) Calculate distance for all points within that classification, sum them up into one value
            temp = [np.linalg.norm(points - self.centroids[centroid]) for centroid in self.centroids]
            localSum = sum(temp)
            
            # Add localSum to totalSum
            totalSum += localSum
            
            # Append totalSum (objective value of k including all those before it)
            self.objectiveDict[classification].append(totalSum)
            
        self.objectivelst = list(self.objectiveDict.values()) 
        
        for sublist in self.objectivelst:
            for item in sublist:
                self.flatList.append(item)
                
        print("SUCCESSIVE OBJECTIVE VALUES: ", self.flatList)
#-----------------------------------------------------------------------------------------------------------------
    def pred(self, coordinatesArr):
        distances = [np.linalg.norm(coordinatesArr - self.centroids[centroid] for centroid in self.centroids)]
        classification = distances.index(min(distances))
        return classification
#-----------------------------------------------------------------------------------------------------------------

In [ ]:
# HELPER METHOD: Execute Strategy No.1 Exclusively
def approach1_execute(X1, a):
    # APPROACH 1, For Loop To Iterate from 2-10 Clusters, 2 runs for the whole thing
    for i in range(2,11):
        print("Run #", a)
        print("REGULAR K MEANS: Clusters = ", i)
        
        # (!!!) K Means 1 and Strat 1
        km = K_Means1(i)
        km.centroidInit_Init1(X1)
        
        # GRAPH: Objective Function per y starts here
        # You need 1 because you NEED to start out with the initial point before second cluster
        locX = np.arange(1, 11, 1)
        
        for iter in range(len(km.flatList)):
            plt.scatter(locX[iter], km.flatList[iter], s = 130, marker = "x")
            
        plt.xlabel("S1: Number Of Clusters, K")
        plt.ylabel("S1: Objective Function Value")
        plt.show()
        #------------------------------------------------------------------------------------------
        # GRAPH: Scatterplot starts here
        colors = 10*["darkorange", "deeppink", "springgreen", "r", "g", "c", "b", "k", "m", "y"]

        for centroid in km.centroids:
            plt.scatter(km.centroids[centroid][0], km.centroids[centroid][1], s = 130, marker = "x")

        for classification in km.classes:
            color = colors[classification]
            for xCoordinate, yCoordinate in km.classes[classification]:
                plt.scatter(xCoordinate, yCoordinate, color = color, s = 30)
        plt.xlabel("S1: X")
        plt.ylabel("S1: Y")
        plt.show()
        print("-------------------------------------------------------------------")

### INITIALIZATION 2: K++ 

In [ ]:
class K_Means2:
    # Constructor
    def __init__(self, k =3, tolerance = 0.0001, max_iterations = 500):
        self.k = k
        self.tolerance = tolerance
        self.max_iterations = max_iterations
#-----------------------------------------------------------------------------------------------------------------    
    # Distance Metric we use
    def Euclidean_distance(feat_one, feat_two):
        squared_distance = 0
        
        # Lengths of two features must be the same
        for i in range(len(feat_one)):
            squared_distance += (feat_one[i] - feat_two[i])**2
        
        ed = sqrt(squared_distance)
        return ed;
#-----------------------------------------------------------------------------------------------------------------    
    # INITIALIZATION 2: Space Out Centroids
    def centroidInit_Init2(self, coordinatesArr):
        self.centroids = {}
        self.objectivelst = []
        self.flatList = []
        
        #(!!!)
        self.objectiveDict = {}
        
        yValueObjective = 0
        
        #----------------------------------------------------------------------------------------
        # STEP 1: Randomize Initial Centroid, Calculate Subsequent Centroids

        # (!!!) NEED TO ITERATE RESPECTIVE TO SELF.K to properly loop through amount of centroids
        
        # RANDOMNESS LIBRARY METHOD: Alternative Way to Randomize Initial Centroids, using Python Randomness Library
        self.centroids[0] = random.choice(coordinatesArr)
            
        for z in range(self.k):
            
            centroidCounter = 0
            masterAvgDict = {}
            
            # This just calculates the distance of ONE point to EVERY CENTROID IN THE LIST
                # You need to do that for ALL of the points within coordinatesArr
            for points in coordinatesArr: 
                # clustDist is a "Mini-Array" respective to ONE POINT
                clustDist = [np.linalg.norm(points - self.centroids[centroid]) for centroid in self.centroids]

                # STEP 1B: Take the AVERAGE of ALL THE VALUES within miniarray clustDist
                pointAverageDist = np.average(clustDist)                

                # STEP 1C: Append that to a master dictionary
                masterAvgDict[centroidCounter] = pointAverageDist
                centroidCounter += 1

            # STEP 1D: Within this given iteration, PICK THE MAXIMUM VALUE as your NEXT CENTROID
            # Retrieve key of maximum average value from masterAvgDict (holds average distance of all points to all current centroids)
            candidateCentroidKey = max(masterAvgDict, key=masterAvgDict.get)
            self.centroids[z] = coordinatesArr[candidateCentroidKey]
        #----------------------------------------------------------------------------------------
        # STEP 2: Begin Iterations
        for i in range(self.max_iterations):
            
            # Dictionary to store points depending on {classification: coordinates}
            self.classes = {}
            
            #(!!!) localSum to sum all the values 
            localSum = 0 
            
            # Initialize Null Dictionary for classes and objective dictionary values
            for i in range(self.k):
                self.classes[i] = []
                self.objectiveDict[i] = []
            
            # STEP 3: Calculate Euclidean Distance Between Point And Cluster
            for points in coordinatesArr: 
                # 'points' have (x, y) form, calculate distance with respect to EACH centroid
                distances = [np.linalg.norm(points - self.centroids[centroid]) for centroid in self.centroids]
                
                # Take the MINIMUM of the distance of EVERY SINGLE POINT respective to that node
                classification = distances.index(min(distances))
                self.classes[classification].append(points)

            previous = dict(self.centroids)

            # STEP 4: Recalcualte Centroids For Next Run
            for classification in self.classes:
                self.centroids[classification] = np.average(self.classes[classification], axis = 0)
                
            isOptimal = True
            
            # STEP 5: Break Out Of Main Loop If Results Are Optimal
            for centroid in self.centroids:
                original_centroid = previous[centroid]
                curr = self.centroids[centroid]
                
                if np.sum((curr - original_centroid)/original_centroid * 100.00) > self.tolerance:
                    isOptimal = False
                    
            if isOptimal == True:
                break
                
        # STEP 6: Plot Objective Function For Each k
        totalSum = 0
        
        # Iterate through every class 
        for classification in self.classes:
            
            # (!!!) Calculate distance for all points within that classification, sum them up into one value
            temp = [np.linalg.norm(points - self.centroids[centroid]) for centroid in self.centroids]
            localSum = sum(temp)
            
            # Add localSum to totalSum
            totalSum += localSum
            
            # Append totalSum (objective value of k including all those before it)
            self.objectiveDict[classification].append(totalSum)
            
        self.objectivelst = list(self.objectiveDict.values()) 
        
        for sublist in self.objectivelst:
            for item in sublist:
                self.flatList.append(item)
                print("ITEM APPENDED: ", item)
        
        print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

In [ ]:
# HELPER METHOD: Execute Initialization No.2 Exclusively
def approach2_execute(X2, b):
    # For Loop To Iterate from 2-10 Clusters, 2 runs for the whole thing
    for i in range(2,11):
        print("Run #", b)
        print("STRATEGY # 2: Clusters = ", i)
        
        km = K_Means2(i)
        km.centroidInit_Init2(X2)
        
        # GRAPH: Objective Function per y starts here
        # You need 1 because you NEED to start out with the initial point before second cluster
        locX = np.arange(1, 11, 1)
        
        for iter in range(len(km.flatList)):
            plt.scatter(locX[iter], km.flatList[iter], s = 130, marker = "x")
            
        plt.xlabel("S2: Number Of Clusters, K")
        plt.ylabel("S2: Objective Function Value")
        plt.show()
        #------------------------------------------------------------------------------------------
        # GRAPH: Scatterplot starts here
        colors = 10*["darkorange", "deeppink", "springgreen", "r", "g", "c", "b", "k", "m", "y"]

        for centroid in km.centroids:
            print("TYPE OF km.centroids[centroid][0]: ", type(km.centroids[centroid][0]))
            print("TYPE OF km.centroids[centroid][1]: ", type(km.centroids[centroid][1]))
            plt.scatter(km.centroids[centroid][0], km.centroids[centroid][1], s = 130, marker = "x")

        for classification in km.classes:
            color = colors[classification]
            for xCoordinate, yCoordinate in km.classes[classification]:
                plt.scatter(xCoordinate, yCoordinate, color = color, s = 30)

        plt.show()
        print("-------------------------------------------------------------------")

In [ ]:
def main():
    X = coordinatesArr

    # REGULAR K MEANS, For Loop To Iterate from 2-10 Clusters, 2 runs for the whole thing
    for a in range(1,3):
        approach1_execute(X, a)
        print("===================================================================")
    
    # K++ MEANS, For Loop To Iterate from 2-10 Clusters, 2 runs for the whole thing
    for b in range(1,3):
        approach2_execute(X, b)
        print("===================================================================")

if __name__ == "__main__":
    main()